In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 300
rcParams["figure.dpi"] = 300

from celerite import plot_setup
plot_setup.setup(auto=False)

In [ ]:
import kplr
import copy
import pickle
import corner
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from celerite.plot_setup import setup, get_figsize, COLORS

import celerite
from celerite import terms

from scipy.ndimage.filters import gaussian_filter

import emcee3
from emcee3 import autocorr

from astropy.stats import LombScargle

In [ ]:
class AsteroTerm(terms.Term):

    parameter_names = (
        "log_S_g", "log_omega_g", "log_nu_max", "log_delta_nu",
        "epsilon", "log_A", "log_Q", "log_W",
    )

    def __init__(self, *args, **kwargs):
        self.nterms = int(kwargs.pop("nterms", 2))
        super(AsteroTerm, self).__init__(*args, **kwargs)

    def get_complex_coefficients(self, params):
        (log_S_g, log_omega_g, log_nu_max, log_delta_nu,
         epsilon, log_A, log_Q, log_W) = params
        alpha = np.exp(log_S_g + log_omega_g) / np.sqrt(2.0)
        beta = np.exp(log_omega_g) / np.sqrt(2.0)
        Q = 0.5 + np.exp(log_Q)
        j = np.arange(-self.nterms, self.nterms+1, 1)
        delta = j*np.exp(log_delta_nu) + epsilon
        omega = 2*np.pi * (np.exp(log_nu_max) + delta)
        S = np.exp(log_A - 0.5*delta**2*np.exp(2*log_W)) / Q**2
        return (
            np.append(alpha, S*omega*Q),
            np.append(alpha, S*omega*Q/np.sqrt(4*Q*Q-1)),
            np.append(beta, 0.5*omega/Q),
            np.append(beta, 0.5*omega/Q*np.sqrt(4*Q*Q-1)),
        )

    def get_envelope(self, omega):
        delta = omega/(2*np.pi) - np.exp(self.log_nu_max)
        return np.sqrt(2/np.pi)*np.exp(self.log_A -
                                       0.5*delta**2*np.exp(2*self.log_W))

    def get_terms(self):
        coeffs = self.get_complex_coefficients()
        return [terms.ComplexTerm(*(np.log(args))) for args in zip(*coeffs)]

    def get_freqs(self):
        j = np.arange(-self.nterms, self.nterms+1, 1)
        delta = j*np.exp(self.log_delta_nu) + self.epsilon
        return np.exp(self.log_nu_max) + delta

    def log_prior(self):
        lp = super(AsteroTerm, self).log_prior()
        if not np.isfinite(lp):
            return lp
        return lp - 0.5 * self.epsilon**2

In [ ]:
# Factor to convert between day^-1 and uHz
uHz_conv = 1e-6 * 24 * 60 * 60

# Download the data for a giant star from MAST
kicid = 11615890
client = kplr.API()
star = client.star(kicid)

x = []
y = []
yerr = []

for lc in star.get_light_curves():
    data = lc.read()
    x0 = data["TIME"]
    y0 = data["PDCSAP_FLUX"]
    m = (data["SAP_QUALITY"] == 0) & np.isfinite(x0) & np.isfinite(y0)
    x.append(x0[m])
    mu = np.median(y0[m])
    y.append((y0[m] / mu - 1.0) * 1e6)
    yerr.append(1e6 * data["PDCSAP_FLUX_ERR"][m] / mu)

x = np.concatenate(x)
y = np.concatenate(y)
yerr = np.concatenate(yerr)

inds = np.argsort(x)
x = np.ascontiguousarray(x[inds], dtype=float)
y = np.ascontiguousarray(y[inds], dtype=float)
yerr = np.ascontiguousarray(yerr[inds], dtype=float)

# Plot the light curve.
fig, ax = plt.subplots(1, 1, figsize=get_figsize())
ax.plot(x, y, "k", rasterized=True)
ax.set_xlim(x.min(), x.max())
ax.set_ylim(np.std(y) * np.array([-5.0, 5.0]))
ax.set_xlabel("time [KBJD]")
ax.set_ylabel("relative flux [ppm]")
ax.xaxis.set_major_locator(plt.MaxNLocator(4))

In [ ]:
# Define a frequency grid for the periodogram
freq_uHz = np.linspace(1, 300, 100000)
freq = freq_uHz * uHz_conv

# Compute the periodogram on the full dataset
model = LombScargle(x, y)
power_all = model.power(freq, method="fast", normalization="psd")
power_all *= uHz_conv / len(x)  # Convert to ppm^2/uHz

# Select a subset of the data
np.random.seed(1234)
n = int(30 * 48)
n0 = np.random.randint(len(x)-n-1)
fit_x, fit_y, fit_yerr = x[n0:n0+n], y[n0:n0+n], yerr[n0:n0+n]
print("Range in subset of data: {0:.1f} days".format(fit_x.max()-fit_x.min()))
print("Fraction of full dataset: {0:.1f}%".format(100 * n / len(x)))

# Compute the periodogram on the subset
model = LombScargle(fit_x, fit_y)
power_some = model.power(freq, method="fast", normalization="psd")
power_some *= uHz_conv / len(fit_x)  # Convert to ppm^2/uHz

# Remove background from periodograms
def estimate_background(x, y, log_width=0.005):
    count = np.zeros(len(x), dtype=int)
    bkg = np.zeros_like(x)
    x0 = np.log10(x[0])
    while x0 < np.log10(x[-1]):
        m = np.abs(np.log10(x) - x0) < log_width
        bkg[m] += np.median(y[m])
        count[m] += 1
        x0 += 0.5 * log_width
    return bkg / count
bkg_all = estimate_background(freq_uHz, power_all)
bkg_some = estimate_background(freq_uHz, power_some)

In [ ]:
# Compute $\nu_\mathrm{max}$ and $\Delta \nu$ from the full dataset
for name, ps in zip(("subset of data", "all data"),
                    (power_some-bkg_some, power_all-bkg_all)):
    # Compute the smoothed power spectrum
    df = freq_uHz[1] - freq_uHz[0]
    smoothed_ps = gaussian_filter(ps, 10 / df)

    # And the autocorrelation function of a lightly smoothed power spectrum
    acor_func = autocorr.function(gaussian_filter(ps, 0.5 / df))
    lags = df*np.arange(len(acor_func))
    acor_func = acor_func[lags < 30]
    lags = lags[lags < 30]

    # Find the peaks
    def find_peaks(z):
        peak_inds = (z[1:-1] > z[:-2]) * (z[1:-1] > z[2:])
        peak_inds = np.arange(1, len(z)-1)[peak_inds]
        peak_inds = peak_inds[np.argsort(z[peak_inds])][::-1]
        return peak_inds

    peak_freqs = freq_uHz[find_peaks(smoothed_ps)]
    nu_max = peak_freqs[peak_freqs > 5][0]

    # Expected delta_nu: Stello et al (2009)
    dnu_expected = 0.263 * nu_max ** 0.772
    peak_lags = lags[find_peaks(acor_func)]
    delta_nu = peak_lags[np.argmin(np.abs(peak_lags - dnu_expected))]
    print("{0}: nu_max = {1}, delta_nu = {2}".format(name, nu_max, delta_nu))

In [ ]:
# Parameter bounds
bounds = dict((n, (-15, 15)) for n in AsteroTerm.parameter_names)
bounds["log_nu_max"] = np.log(np.array([130.0, 190.0])*uHz_conv)
bounds["log_delta_nu"] = np.log(np.array([12.5, 13.5])*uHz_conv)
bounds["log_W"] = (-3, 3)

# Set up the GP model
kernel = AsteroTerm(
    log_S_g=np.log(np.var(y)),
    log_omega_g=2.0,
    log_nu_max=np.log(nu_max*uHz_conv),
    log_delta_nu=np.log(delta_nu*uHz_conv),
    epsilon=0.0,
    log_A=np.log(np.var(y)),
    log_Q=5.0,
    log_W=-1.0,
    bounds=bounds,
    nterms=2,
)
kernel += terms.JitterTerm(
    log_sigma=np.log(np.median(np.abs(np.diff(fit_y)))),
    bounds=[(-15, 15)]
)
gp = celerite.GP(kernel)
gp.compute(fit_x, fit_yerr)
print("Initial log-likelihood: {0}".format(gp.log_likelihood(fit_y)))
print(gp.get_parameter_dict(include_frozen=True))

# The objective function for optimization
def nll(params):
    gp.set_parameter_vector(params)
    ll = gp.log_likelihood(fit_y, quiet=True)
    if not np.isfinite(ll):
        return 1e10
    return -ll

def grad_nll(params):
    gp.set_parameter_vector(params)
    return -gp.grad_log_likelihood(fit_y)[1]

# Grid initialize
print("Running a grid of optimizations...")
gp.kernel.thaw_all_parameters()
initial = np.array(gp.get_parameter_vector())

def get_ml_params(log_nu_max):
    gp.set_parameter_vector(initial)
    gp.kernel.set_parameter("terms[0]:log_nu_max", log_nu_max)
    gp.kernel.set_parameter(
        "terms[0]:log_delta_nu",
        np.log(0.263 * (np.exp(log_nu_max)/uHz_conv) ** 0.772 * uHz_conv)
    )
    p0 = gp.get_parameter_vector()
    bounds = gp.get_parameter_bounds()
    r = minimize(nll, p0, method="L-BFGS-B", bounds=bounds)
    gp.set_parameter_vector(r.x)
    return r.fun, r.x

with emcee3.pools.InterruptiblePool() as pool:
    results = list(sorted(pool.map(
        get_ml_params, gp.kernel.terms[0].log_nu_max + np.linspace(-0.05, 0.05, 5)
    ), key=lambda o: o[0]))
gp.set_parameter_vector(results[0][1])
print(gp.get_parameter_dict(include_frozen=True))

In [ ]:
# Use more modes in the MCMC:
gp.kernel.terms[0].nterms = 3

fig, ax = plt.subplots(1, 1, figsize=get_figsize())
ax.plot(freq_uHz, power_all, "k", alpha=0.8, rasterized=True)
ax.plot(freq_uHz, gp.kernel.get_psd(2*np.pi*freq) * uHz_conv / (2*np.pi),
        alpha=0.5, rasterized=True)
ax.set_xlabel("frequency [$\mu$Hz]")
ax.set_ylabel("power [$\mathrm{ppm}^2\,\mu\mathrm{Hz}^{-1}$]")
ax.set_yscale("log")

In [ ]:
# Set up the probabilistic model for sampling
def log_prob(p):
    gp.set_parameter_vector(p)
    lp = gp.log_prior()
    if not np.isfinite(lp):
        return -np.inf
    ll = gp.log_likelihood(fit_y)
    if not np.isfinite(ll):
        return -np.inf
    return ll + lp

# Initialize and set bounds
ndim, nwalkers = gp.vector_size, 32
initial_samples = \
    gp.get_parameter_vector() + 1e-5 * np.random.randn(nwalkers, ndim)

names = gp.get_parameter_names()
ind_nu_max = names.index("kernel:terms[0]:log_nu_max")
ind_delta_nu = names.index("kernel:terms[0]:log_delta_nu")

# Save the current state of the GP and data
with open("astero-{0}.pkl".format(kicid), "wb") as f:
    pickle.dump((
        gp, fit_y, freq, power_all, power_some, len(x),
    ), f, -1)

In [ ]:
# Define a custom proposal
def astero_move(rng, x0):
    x = np.array(x0)
    f = 2.0 * (rng.rand(len(x)) < 0.5) - 1.0
    x[:, ind_nu_max] = np.log(np.exp(x[:, ind_nu_max]) +
                              f * np.exp(x[:, ind_delta_nu]))
    return x, np.zeros(len(x))

# The sampler will use a mixture of proposals
sampler = emcee3.Sampler([
    emcee3.moves.StretchMove(),
    emcee3.moves.DEMove(1e-3),
    emcee3.moves.KDEMove(),
    emcee3.moves.MHMove(astero_move),
], backend=emcee3.backends.HDFBackend("astero-{0}.h5".format(kicid)))

# Sample!
with emcee3.pools.InterruptiblePool() as pool:
    ensemble = emcee3.Ensemble(emcee3.SimpleModel(log_prob), initial_samples,
                               pool=pool)
    ensemble = sampler.run(ensemble, 20000, progress=True)

In [ ]:
c = sampler.get_coords()
plt.plot(c[:, :, ind_nu_max], alpha=0.3);

In [ ]:
samples = sampler.get_coords(discard=5000, flat=True)
log_probs = sampler.get_log_probability(discard=5000, flat=True)

In [ ]:
names = list(gp.get_parameter_names())
for i in range(len(names)):
    name = names[i].split(":")[-1]
    if name.startswith("log"):
        name = "log("+name[4:]+")"
    names[i] = name.replace("_", " ")

In [ ]:
measurement_var = np.median(gp._yerr**2)
white_noise_all = measurement_var * uHz_conv / len(y)
white_noise_some = measurement_var * uHz_conv / len(fit_y)

In [ ]:
# Compute the model predictions
time_grid = np.linspace(0, 1.4, 5000)
n = 1000
psds = np.empty((n, len(freq)))
acors = np.empty((n, len(time_grid)))
for i, j in enumerate(np.random.randint(len(samples), size=n)):
    s = samples[j]
    gp.set_parameter_vector(s)
    psds[i] = gp.kernel.get_psd(2*np.pi*freq)
    acors[i] = gp.kernel.get_value(time_grid)

# Get the median modes
gp.set_parameter_vector(samples[np.argmax(log_probs)])
peak_freqs = gp.kernel.terms[0].get_freqs()

In [ ]:
# Plot constraints on nu-max and delta-nu
i = [names.index("log(nu max)"), names.index("log(delta nu)")]
s = np.exp(samples[:, i])/uHz_conv
nu_max_pub = 171.94, 3.62
delta_nu_pub = 13.28, 0.29
fig = corner.corner(s, smooth=0.7, smooth1d=1.0,
                    labels=[r"$\nu_\mathrm{max}$", r"$\Delta \nu$"])
fig.axes[2].errorbar(nu_max_pub[0], delta_nu_pub[0],
                     xerr=nu_max_pub[1], yerr=delta_nu_pub[1],
                     fmt=".", color=COLORS["MODEL_1"], capsize=0,
                     lw=2, mec="none")
ax = fig.axes[0]
y = np.mean(ax.get_ylim())
ax.errorbar(nu_max_pub[0], y, xerr=nu_max_pub[1],
            fmt=".", color=COLORS["MODEL_1"], capsize=0,
            lw=2, mec="none")

ax = fig.axes[3]
y = np.mean(ax.get_ylim())
ax.errorbar(delta_nu_pub[0], y, xerr=delta_nu_pub[1],
            fmt=".", color=COLORS["MODEL_1"], capsize=0,
            lw=2, mec="none")

fig.savefig("astero-corner.pdf", bbox_inches="tight")

In [ ]:
fig = corner.corner(samples, smooth=0.7, smooth1d=1.0, labels=names)

In [ ]:
from scipy.linalg import cho_solve, cho_factor

p0 = gp.get_parameter_vector()
fast_timing = %timeit -o log_prob(p0)

def _time_this():
    K = gp.get_matrix(include_diagonal=True)
    factor = cho_factor(K, overwrite_a=True)
    ld = 2.0 * np.sum(np.log(np.diag(factor[0])))
    ll = -0.5*(np.dot(fit_y, cho_solve(factor, fit_y))+ld) + gp.log_prior()

slow_timing = %timeit -o _time_this()

In [ ]:
chain = sampler.get_coords(discard=5000)[:, :, [ind_nu_max, ind_delta_nu]]
tau = np.mean(autocorr.integrated_time(np.mean(chain, axis=1), c=5))
neff = len(samples) / tau
tau, neff

In [ ]:
import json
c = gp.kernel.coefficients
with open("astero.json", "w") as f:
    json.dump(dict(
        N=len(fit_x),
        J=len(c[0]) + len(c[2]),
        tau=tau,
        neff=neff,
        time=fast_timing.average,
        direct_time=slow_timing.average,
        nwalkers=nwalkers,
        nburn=5000,
        nsteps=15000,
        ndim=int(ndim),
    ), f)

In [ ]:
name_map = {
    'kernel:terms[0]:log_S_g': "$\ln(S_g/\mathrm{ppm}^2)$",
    'kernel:terms[0]:log_omega_g': "$\ln(\omega_g/\mathrm{day}^{-1})$",
    'kernel:terms[0]:log_nu_max': "",
    'kernel:terms[0]:log_delta_nu': "",
    'kernel:terms[0]:epsilon': "",
    'kernel:terms[0]:log_A': "$\ln(A/\mathrm{ppm}^2\,\mathrm{day})$",
    'kernel:terms[0]:log_Q': "$\ln(Q)$",
    'kernel:terms[0]:log_W': "$\ln(W/\mathrm{day}^{-1})$",
    'kernel:terms[1]:log_sigma': "$\ln(\sigma/\mathrm{ppm})$",
}
params = list(zip(
    (name_map[n] for n in gp.get_parameter_names()),
    gp.get_parameter_bounds()
))
params[ind_nu_max] = r"$\ln(\nu_\mathrm{max}/\mu\mathrm{Hz})$", ["\ln(130)", "\ln(190)"]
params[ind_delta_nu] = r"$\ln(\Delta \nu/\mu\mathrm{Hz})$", ["\ln(12.5)", "\ln(13.5)"]
params[ind_delta_nu+1] = "$\epsilon/\mathrm{day}^{-1}$", ["$\mathcal{N}(0,\,1)$"]
with open("astero-params.json", "w") as f:
    json.dump(params, f)

In [ ]:
# Make comparison plot
fig, axes = plt.subplots(3, 1, sharex=True, sharey=True,
                         figsize=get_figsize(2.5, 2))

axes[0].plot(freq_uHz, power_all, "k", rasterized=True)
axes[0].plot(freq_uHz, gaussian_filter(power_all, 150),
             color=COLORS["MODEL_2"], rasterized=True)
axes[0].axhline(white_noise_all)

axes[1].plot(freq_uHz, power_some, "k", rasterized=True)
axes[1].plot(freq_uHz, gaussian_filter(power_some, 450),
             color=COLORS["MODEL_2"], rasterized=True)
axes[1].axhline(white_noise_some)

q = np.percentile(uHz_conv/(2*np.pi)*psds, [16, 50, 84], axis=0)
axes[2].fill_between(freq_uHz, q[0], q[2], color="k", alpha=0.3,
                     rasterized=True)
axes[2].plot(freq_uHz, q[1], "k", alpha=0.8, rasterized=True)
axes[2].axhline(white_noise_some)

labels = [
    "periodogram estimator\n4 years of data",
    "periodogram estimator\n1 month of data",
    "posterior inference\n1 month of data",
]
for ax, label in zip(axes, labels):
    ax.set_yscale("log")
    for f in peak_freqs / uHz_conv:
        ax.plot([f, f], [2e2, 3e2], "k", lw=0.5)
    ax.annotate(label, xy=(1, 1), xycoords="axes fraction",
                ha="right", va="top",
                xytext=(-5, -5), textcoords="offset points",
                fontsize=12)
    ax.set_ylabel("power [$\mathrm{ppm}^2\,\mu\mathrm{Hz}^{-1}$]")

axes[2].set_xlabel("frequency [$\mu$Hz]")
axes[2].set_xlim(freq_uHz.min(), freq_uHz.max())
axes[2].set_ylim(1e-3, 4e2)

fig.savefig("astero-comp.pdf", bbox_inches="tight", dpi=300)

In [ ]:
gp.set_parameter_vector(samples[np.argmax(log_probs)])
t0 = fit_x.min()
x = np.linspace(t0+9.5, t0+20.5, 1000)
mu, var = gp.predict(fit_y, x, return_var=True)
std = np.sqrt(var)

pred_mu, pred_var = gp.predict(fit_y, return_var=True)

In [ ]:
fig = plt.figure(figsize=plot_setup.get_figsize(1, 2))

ax1 = plt.subplot2grid((3, 2), (0, 0), rowspan=2)
ax2 = plt.subplot2grid((3, 2), (2, 0), rowspan=1)
fig.subplots_adjust(hspace=0, wspace=0.1)

ax1.errorbar(fit_x - t0, fit_y, yerr=fit_yerr, fmt=".k",
             lw=0.5, ms=3, rasterized=True)
ax1.plot(x - t0, mu, lw=0.75, rasterized=True)
ax1.fill_between(x-t0, mu+std, mu-std, alpha=0.5, edgecolor="none", zorder=100)
ax1.set_xticklabels([])

ax1.annotate("N = {0}".format(len(fit_x)), xy=(0, 1),
             xycoords="axes fraction",
             xytext=(5, -5), textcoords="offset points",
             ha="left", va="top")

sig = np.sqrt(fit_yerr**2 + pred_var)
ax2.errorbar(fit_x - t0, fit_y - pred_mu, yerr=sig, fmt=".k",
             ms=3, lw=0.5, rasterized=True)
ax2.axhline(0.0, color="k", lw=0.75)

ax1.set_ylim(-750, 750)
ax1.set_xlim(9.5, 20.5)
ax2.set_ylim(-245, 245)
ax2.set_xlim(9.5, 20.5)

ax2.set_xlabel("time [day]")
ax1.set_ylabel("relative flux [ppm]")
ax2.set_ylabel("residuals")

for ax in [ax1, ax2]:
    ax.yaxis.set_label_coords(-0.22, 0.5)
    
fig.savefig("astero.pdf", bbox_inches="tight", dpi=300)